In [2]:
__author__ = "zeshi"
from __future__ import print_function

import os
import time
from datetime import datetime, timedelta
from multiprocessing import Pool, cpu_count
import mysql.connector
from mysql.connector import errorcode
from mysqldb_level0 import populate_data_server
from level0_2_level1 import level0_to_level1_data_merge

# rsync data from webserver and transfer data from local to local
os.system("python rsync_ssh.py")
os.system("python tmp_to_server_data.py")

print("Finished transfer data from webserver to compserver!")

# Query site_names from mysql and populate server data into level_0 table
cnx = mysql.connector.connect(user = "root", password = "root", database = "ar_data")
cursor = cnx.cursor()
try:
    cursor.execute("SELECT site_name FROM sites")
    site_names = cursor.fetchall()
except mysql.connector.Error as err:
    print(err)
cursor.close()
cnx.close()

site_names_list = []

for site_name in site_names:
    site_names_list.append(site_name[0])

# Initialize parallel processing
pool = Pool(processes=cpu_count())
# Mapping populate data server with the site_names_list
pool.map(populate_data_server, site_names_list)
pool.close()
pool.join()

# Query site_name and number of node at each site
cnx = mysql.connector.connect(user = "root", password = "root", database = "ar_data")
cursor = cnx.cursor()
try:
    cursor.execute("SELECT site_name, num_of_nodes FROM sites")
    sites_infos = cursor.fetchall()
except mysql.connector.Error as err:
    print(err)
cursor.close()
cnx.close()

def merge0_to_1_parallel(site_info):
    starting_time = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0) - timedelta(days=1)
    ending_time = datetime.today().replace(hour=0, minute=0, second=0, microsecond=0)
    site_name = site_info[0]
    site_num_of_nodes = site_info[1]
    for node_id in range(1, site_num_of_nodes + 1):
        level0_to_level1_data_merge(site_name, node_id, datetime_range_interupt=(starting_time, ending_time))
    
# Initizalize parallel processing
pool = Pool(processes=cpu_count())
# Merge level_0 data to level_1 data
pool.map(merge0_to_1_parallel, sites_infos)
pool.close()
pool.join()

Finished transfer data from webserver to compserver!
Start populating server data into mysql at Alpha
Start populating server data into mysql at Bear_Trap
Start populating server data into mysql at Caples_Lk
Start populating server data into mysql at DollyRice
Start populating server data into mysql at Duncan_Pk
Start populating server data into mysql at Mt_Lincoln
Start populating server data into mysql at Echo_Pk
Start populating server data into mysql at Onion_Ck
Start populating server data into mysql at Owens_Camp
Start populating server data into mysql at Robbs_Saddle
Start populating server data into mysql at Schneiders
Start populating server data into mysql at Talbot_Camp
Start populating server data into mysql at Van_Vleck
Alpha: 383c19
Bear_Trap: 3ff6e4
Caples_Lk: 383bea
Duncan_Pk: 382a73
Mt_Lincoln: 381f53
Echo_Pk: 382a77
Onion_Ck: 602ba0
Robbs_Saddle: 383343
Schneiders: 383ad8
Van_Vleck: 0038a3
Alpha: 383abe
No node found to mac address: 3ff6e4Bear_Trap
Caples_Lk: 383ad8
N

NameError: name 'site_infos' is not defined

In [ ]:
def long_term_update(starting_time, real_ending_time):
    # Query site_infos, site_name and number of nodes
    cnx = mysql.connector.connect(user = "root", password = "root", database = "ar_data")
    cursor = cnx.cursor()
    try:
        cursor.execute("SELECT site_name, num_of_nodes FROM sites")
        sites_infos = cursor.fetchall()
    except mysql.connector.Error as err:
        print(err)
    cursor.close()
    cnx.close()
    
    # ending_time is 7 days later than starting date
    ending_time = starting_time + timedelta(days=7)
    while ending_time <= real_ending_time:
        print(starting_time, ending_time)
        for site_info in sites_infos:
            site_name = site_info[0]
            site_num_of_nodes = site_info[1]
            for node_id in range(1, site_num_of_nodes + 1):
                level0_to_level1_data_merge(site_name, node_id, datetime_range_interupt=(starting_time, ending_time))
        starting_time += timedelta(days=7)
        ending_time += timedelta(days=7)